In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
!unzip "/content/drive/MyDrive/fruits.zip"

In [ ]:
import tensorflow as tf

In [ ]:
#Data and directory summary after unnecessary data has been deleted
import os
for dirpath,dirname,filenames in os.walk("train"):
  print(f"There are {len(dirname)} directories and {len(filenames)} images in {dirpath}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
import matplotlib.image as mpimg
import random
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import load_img, img_to_array
import pathlib
import numpy as np

In [ ]:
data_dir = pathlib.Path("/content/train/train")
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

In [ ]:
!pip install split-folders
import splitfolders
splitfolders.ratio('/content/train/train',output="/content/Fruits",seed=42,ratio=(0.8,0.1,0.1))

In [ ]:
#setting up the path for training,validation and testing directories
train_dir="/content/Fruits/train"
test_dir="/content/Fruits/test"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
augment_datagen = ImageDataGenerator(rotation_range=20,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         rescale=1/255.,
                         shear_range=0.2,
                         zoom_range=0.2,
                         horizontal_flip=True,)
test_datagen=ImageDataGenerator(rescale=1/255.)
train_data=augment_datagen.flow_from_directory(train_dir,
                                             target_size=(100,100),
                                             batch_size=64,
                                             class_mode="categorical",
                                             shuffle=True)
test_data=test_datagen.flow_from_directory(test_dir,
                                           target_size=(100,100),
                                           batch_size=64,
                                           class_mode="categorical")

In [ ]:
import matplotlib.pyplot as plt
def plot_loss_curves(history):

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam


# Load the pre-trained VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(100,100,3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

VGG16_model=Sequential()
VGG16_model.add(base_model)


VGG16_model.add(Flatten())
VGG16_model.add(Dense(512, activation='relu'))
VGG16_model.add(Dropout(0.5))
VGG16_model.add(Dense(33, activation='softmax'))

# Print the model summary
VGG16_model.summary()

# Compile the model
VGG16_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history_VGG16 = VGG16_model.fit(train_data,
                          epochs=1,
                          steps_per_epoch=len(train_data),
                          validation_data=test_data,
                          validation_steps=len(test_data))

In [ ]:
VGG16_model.evaluate(test_data)

In [ ]:
from PIL import Image
def load_and_prep_image(filename, img_shape=100):
  """
  Reads an image from filename, turns it into a tensor
  and reshapes it to (img_shape, img_shape, colour_channel).
  """

  # Read in target file (an image)
  img = tf.io.read_file(filename)

  # Decode the read file into a tensor & ensure 3 colour channels
  # (our model is trained on images with 3 colour channels and sometimes images have 4 colour channels)
  img = tf.image.decode_image(img, channels=3)

  # Resize the image (to the same size our model was trained on)
  img = tf.image.resize(img, size = [img_shape, img_shape])

  # Rescale the image (get all values between 0 and 1)
  img = img/255.
  return img


In [ ]:
def pred_and_plot(model, filename, class_names):
  """
  Imports an image located at filename, makes a prediction on it with
  a trained model and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class
  if len(pred[0]) > 1: # check for multi-class
    pred_class = class_names[pred.argmax()] # if more than one output, take the max
  else:
    pred_class = class_names[int(tf.round(pred)[0][0])] # if only one output, round


  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

In [ ]:
pred_and_plot(VGG16_model,"/content/Fruits/test/Cherry/Cherry_100.jpg", class_names)

In [ ]:
VGG16_model.save("FruitDetector.h5")

In [ ]:
loaded_model = tf.keras.models.load_model("FruitDetector.h5")
loaded_model.evaluate(test_data)

In [ ]:
pred_and_plot(VGG16_model,"/content/Banana.jpg", class_names)

In [ ]:
pred_and_plot(VGG16_model,"/content/watermleon.jpg", class_names)

In [ ]:
pred_and_plot(VGG16_model,"/content/Lemon.jpg", class_names)

In [ ]:
pred_and_plot(VGG16_model,"/content/Pomegranate.jpg", class_names)

In [ ]:
pred_and_plot(VGG16_model,"/content/corn_sample.jpg", class_names)

In [ ]:
VGG16_model.save("FruitDetector.h5")

In [ ]:
import tensorflow as tf
loaded_model = tf.keras.models.load_model("FruitDetector.h5")

In [ ]:
from google.colab import files
files.download('FruitDetector.h5')